# LangChain + GitHub Models + Azure OpenAI

LangChain is a modular framework designed to streamline the development and integration of applications utilizing large language models (LLMs) by providing tools for effective prompt engineering, data handling, and complex workflows.

LangChain supports a number of model providers including Azure OpenAI and Cohere. Below find examples of how LangChain can be used with these models provided by the GitHub Models service.


## 1. Install dependencies

In [ ]:
%pip install langchain-openai
%pip install langchain-community
%pip install faiss-cpu
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 2. Constructing LangChain's ChatOpenAI class

Setting up the OpenAI API key and base URL for LangChain is the same as for OpenAI's Python client. You can set the `OPENAI_API_KEY` and `OPENAI_API_BASE` environment variables, or pass them directly to the `ChatOpenAI` class.


In [17]:
from langchain_openai import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv()

if not os.getenv("GITHUB_TOKEN"):
    raise ValueError("GITHUB_TOKEN is not set")

os.environ["OPENAI_API_KEY"] = os.getenv("GITHUB_TOKEN")
os.environ["OPENAI_BASE_URL"] = "https://models.inference.ai.azure.com/"

GPT_MODEL = "gpt-4o-mini"

llm = ChatOpenAI(model=GPT_MODEL)

Now you can use the ChatOpenAI class to interact with the OpenAI API. 

In [18]:
from IPython.display import Markdown, display

response = llm.invoke("What is the meaning of life?")
display(Markdown(response.content))

The meaning of life is a profound question that has been explored by philosophers, theologians, scientists, and thinkers throughout history. Different cultures and individuals may offer various interpretations based on their beliefs, experiences, and values. Here are a few perspectives:

1. **Philosophical**: Some philosophers argue that life has no inherent meaning and that individuals must create their own purpose through choices, relationships, and experiences. Existentialists like Jean-Paul Sartre emphasize personal freedom and responsibility.

2. **Religious**: Many religious traditions provide specific meanings of life based on the belief in a higher power or an ultimate purpose. For example, in Christianity, life may be seen as a journey to know and serve God, while in Buddhism, it may involve the pursuit of enlightenment and the alleviation of suffering.

3. **Scientific**: From a biological perspective, the purpose of life can be viewed in terms of survival and reproduction. Evolutionary biology suggests that the driving forces of life are the continuation of genes and adaptation to the environment.

4. **Personal**: For many, the meaning of life is derived from personal fulfillment, relationships, and contributions to society. People often find purpose through love, creativity, work, and helping others.

Ultimately, the meaning of life can be a deeply personal exploration, and individuals may find different meanings at various stages of their lives.

## 3. Use a prompt template

LangChain has the concept of a prompt template that allows you parameterize the prompt and fill in the parameters with values. 

In [19]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a fun assistant that uses a lot of emojis."),
    ("user", "{input}")
])

Next, we combine the promt with the ChatOpenAI into a chain:

In [20]:
chain = prompt | llm 

Then we invoke the chain with the input data:

In [21]:
chain.invoke({"input": "when was the first ESC held?"})


AIMessage(content='The first Eurovision Song Contest (ESC) was held on May 24, 1956! 🎤🎶 It took place in Lugano, Switzerland, and featured just seven countries competing. What a fun start to a legendary music competition! 🌍✨', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 30, 'total_tokens': 82, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_d54531d9eb', 'finish_reason': 'stop', 'logprobs': None}, id='run-36a0ab04-7ec9-41f9-b04d-f968f26da376-0', usage_metadata={'input_tokens': 30, 'output_tokens': 52, 'total_tokens': 82, 'input_token_details': {}, 'output_token_details': {}})

## 4. Add a parsing step to your chain

We can also extend the chain to include additional steps, such as parsing the content from the reponse:

In [22]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser

chain.invoke({"input": "when was the first ESC held?"})

'The first Eurovision Song Contest (ESC) was held on May 24, 1956! 🎤🌍 It took place in Lugano, Switzerland, and featured just seven countries competing. What a fun beginning to such a long-running tradition! 🎶✨'

## 5. Use retrieval to ground the model

Now let's ask for some more recent information:

In [23]:
chain.invoke({"input": f"Who won the 2024 ESC?"})

"I don't have information on events that happened after October 2023, so I can't tell you who won the 2024 Eurovision Song Contest. 🌍🎤✨ But I can help you with details about past contests or anything else you might want to know! 🎶💖"

Looks like the model is not up to date with the latest contest. Let's help it out with some content from the wikipedia:

In [24]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Eurovision_Song_Contest_2024")

docs = loader.load()
len(docs[0].page_content)

175390

Due to the limits of the free service, the content retrieved from Wikipedia is too large to be added to the context -- that is ok since we can use a vector store to index the content. For that we need an embedding model which we can use with the OpenAIEmbeddings class. Note that we need to reduce the chunk size due to the token limits of the GitHub Models free service. 

In [25]:
from langchain_openai import OpenAIEmbeddings

EMBEDDINGS_MODEL = "text-embedding-3-small"

# need to constrain the chunk_size to work within the limits of the GitHub Model service
embeddings = OpenAIEmbeddings(model=EMBEDDINGS_MODEL,chunk_size=30)

Now, we will use the [FAISS](https://github.com/facebookresearch/faiss) library to index the vectors for a similarity search.

In [26]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Next, we can create a document chain that will use some context and the user's question to provide an answer:

In [27]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""You are a fun assistant that uses a lot of emojis.
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

We could just provide the context directly to the chain like so:

In [28]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Who won 2024 ESC?",
    "context": [Document(page_content="The top 3 placings in the 2024 Eurovision Song Contest were:\n1. Switzerland\n2. Croatia\n3. Ukraine")]
})

'🎉 The winner of the 2024 Eurovision Song Contest is Switzerland! 🇨🇭✨'

But we really want a retrieval chain that will retrieve the right documents from our vector index and then uses it as context in our prompt like so:

In [29]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

Let's ask this one who won the 2024 ESC:

In [30]:
response = retrieval_chain.invoke({"input": f"Who won the 2024 ESC?"})
print(response["answer"])


🎉🎶 The winner of the Eurovision Song Contest 2024 was Switzerland with the song "The Code," performed by Nemo! 🇨🇭✨🎤 Congrats to them! 🥳🎊


And a few more increasingly detailed questions:

In [31]:
response = retrieval_chain.invoke({"input": "where did the 2024 ESC take place?"})
print(response["answer"])

The 2024 Eurovision Song Contest took place in Malmö, Sweden! 🇸🇪🎶✨


In [32]:
response = retrieval_chain.invoke({"input": "who designed the stage for the 2024 ESC?"})
print(response["answer"])

The stage for the Eurovision Song Contest 2024 was designed by the talented German production designer **Florian Wieder**! 🎨✨ He had previously designed the sets for six other contests, making him a seasoned pro! 🎤🌟


In [33]:
response = retrieval_chain.invoke({"input": "when did the semi-finals for the 2024 ESC take place?"})
print(response["answer"])

The semi-finals for the Eurovision Song Contest 2024 took place on **7 May 2024** and **9 May 2024**! 🎤✨🎶
